In [59]:
# LSST stack imports
from lsst.daf.persistence import Butler
import lsst.afw.display as afw_display
%matplotlib inline
import matplotlib.pyplot as plt
import math
import pandas as pd

# Firefly client imports
from firefly_client import FireflyClient

# Standard libraries in support of Firefly display
from urllib.parse import urlparse, urlunparse, ParseResult
from IPython.display import display, Markdown
import os

In [6]:
##### If you want to edit the data set, tract, and patch yourself, change the numbers below appropriately###
data_set = 'UDEEP'# can be WIDE, DEEP, UDEEP
datadir = '/datasets/hsc/repo/rerun/DM-13666/' + data_set 
data_set = 'deepCoadd_forced_src'
butler = Butler(sample_datadir)
tract = 8765 #8766
patch = '1,2' #'8,3'
#All subsequent data structures will be stored in sets, with the elements of the sets 
# corresponding to the filters you specify here.  So, probably a good idea to remember the order! 
filters = ['HSC-I','HSC-Y'] 
n_filters = len(filters)
data_set = 'deepCoadd_forced_src'

In [19]:
deep_coadds = [butler.get(data_set, tract = tract, patch = patch, dataId={'filter': filter}) for filter in filters]
n_raw_objects = len(deep_coadds[0])
for i in range(len(filters)):
    print ('Number of objects in filter ' + filters[i] + ' = ' + str(len(deep_coadds[i])))
print ('(Note that those numbers should all be the same.)')
#print (deep_coadd_I)
#print (deep_coadd_Y)
deep_coadd_tables = [deep_coadd.asAstropy() for deep_coadd in deep_coadds]

Number of objects in filter HSC-I = 25266
Number of objects in filter HSC-Y = 25266
(Note that those numbers should all be the same.)


In [57]:
#Define all possible flags, and our recommended flags 
flags_to_reject_on_recommended= ['base_PixelFlags_flag_bad', 'base_PixelFlags_flag_cr', 'base_PixelFlags_flag_offimage','base_PixelFlags_flag_edge', 'base_PixelFlags_flag_saturated', 'base_PixelFlags_flag_rejected', 'base_PixelFlags_flag_interpolated']
flags_to_reject_on_all = [ (colname, deep_coadd_tables[0][colname].description) for colname in deep_coadd_tables[0].colnames if '_flag_' in colname]
#print (flags_to_reject_on_all)


In [12]:
print ('Choose the flags on which you wish to reject observations. ')
print ('Here are your options: ')
print (flags_to_reject_on_all)
print ('Here are our suggestions:')
print (flags_to_reject_on_recommended)

Choose the flags on which you wish to reject observations. 
Here are your options: 
['base_PixelFlags_flag_bad', 'base_PixelFlags_flag_cr', 'base_PixelFlags_flag_offimage', 'base_PixelFlags_flag_edge', 'base_PixelFlags_flag_saturated', 'base_PixelFlags_flag_rejected', 'base_PixelFlags_flag_interpolated']
Here are our suggestions:
['base_PixelFlags_flag_bad', 'base_PixelFlags_flag_cr', 'base_PixelFlags_flag_offimage', 'base_PixelFlags_flag_edge', 'base_PixelFlags_flag_saturated', 'base_PixelFlags_flag_rejected', 'base_PixelFlags_flag_interpolated']


In [46]:
######ENTER YOUR CHOICE OF REJECTION FLAGS HERE##### 
flags_to_reject_on = [] #Say ['all'] if you want to use all possible flags 
if len(flags_to_reject_on) == 0: 
    flags_to_reject_on = flags_to_reject_on_recommended[:]
if flags_to_reject_on[0] is 'all':
    flags_to_reject_on_all 

In [27]:
#Do or operation on all flags in individual filter, and then between filter.  
# So if an object has ANY of the desired flags high, that object will be rejected by the master flag 
total_rej_flags_by_filters = [[any([deep_coadd_table[flag][i] for flag in flags_to_reject_on]) for i in range(n_raw_objects)] 
                                              for deep_coadd_table in deep_coadd_tables] 
for i in range(n_filters):
    print ('For filter ' + str(filters[i]) + ', ' + str(len([flag for flag in total_rej_flags_by_filters[i] if flag == False])) + ' good objects were found, based on your chosen flags.')


For filter HSC-I, 15105 good objects were found, based on your chosen flags.
For filter HSC-Y, 10497 good objects were found, based on your chosen flags.


In [33]:
master_rejection_flags = [any([total_rej_flags[i] for total_rej_flags in total_rej_flags_by_filters]) for i in range(n_raw_objects)]
good_objects_by_filter = [deep_coadd_table[[not(flag) for flag in master_rejection_flags]] 
                         for deep_coadd_table in deep_coadd_tables]
print ('Combining them all, we are left with ' + str(len(good_objects_by_filter[0])) + ' good objects. ') 
n_good_objects = len(good_objects_by_filter[0])

Combining them all, we are left with 8242 good objects. 


In [29]:
print ('We have now parsed the given object lists by the specified flags.')
print  ('If you JUST want objects sliced by the given flags, take the good_objects_by_filter array. ')
print ('But if you want to also cut based on whether an object is a star or a galaxy, then continue onward!') 

We have now parsed the given object lists by the specified flags.
If you JUST want objects sliced by the given flags, take the good_objects_by_filter array. 
But if you want to also cut based on whether an object is a star or a galaxy, then continue onward!


In [43]:
extendedness_by_filter = [good_objects['base_ClassificationExtendedness_value'] 
                          for good_objects in good_objects_by_filter]
#Note that is_star is NOT the opposite of the is_galaxy
# This is because the extendness is sometimes nan, which we treat as neither 
#   a star nor a galaxy.

is_star_by_filter = [[extendedness <= 0 for extendedness in extendednesses] 
                      for extendednesses in extendedness_by_filter] 
is_galaxy_by_filter = [[extendedness > 0 for extendedness in extendednesses] 
                      for extendednesses in extendedness_by_filter] 


In [44]:
for i in range(n_filters):
    print ('For filter ' + str(filters[i]) + ', there are: ')
    print (str(len([is_star for is_star in is_star_by_filter[i] if is_star == True])) + ' good stars.')
    print (str(len([is_galaxy for is_galaxy in is_galaxy_by_filter[i] if is_galaxy == True])) + ' good galaxies.')

For filter HSC-I, there are: 
2148 good stars.
4523 good galaxies.
For filter HSC-Y, there are: 
2016 good stars.
4650 good galaxies.


In [49]:
master_is_star = [all([is_star[i] for is_star in is_star_by_filter]) for i in range(n_good_objects)]
master_is_galaxy = [all([is_star[i] for is_star in is_star_by_filter]) for i in range(n_good_objects)]
good_stars_by_filter = [good_objects_by_filter[i][master_is_star] for i in range(n_filters)] 
good_galaxy_by_filter = [good_objects_by_filter[i][master_is_galaxy] for i in range(n_filters)] 


In [50]:
print ('After looking only for those that are deemed to be stars or galaxies in ALL filters, we find that there are: ')
print (str(len(good_stars_by_filter[0])) + ' good stars. ')
print (str(len(good_galaxies_by_filter[0])) + ' good galaxies. ')

After looking only for those that are stars or galaxies in ALL filters, we find that there are: 
946 good stars. 
4523 good galaxies. 


In [60]:
#Now we want to know the fraction of objects rejected by each flag:

#tables = [catalog.asAstropy() for catalog in catalogs]

def format_fraction(table, colname):
    size = len(table)
    fraction = "{}%".format(int(len(table[table[colname]==True])/size*100))
    return fraction

fraction_rejected = [[colname] +  [format_fraction(table, colname) for table in deep_coadd_tables] 
                     for colname, description in flags_to_reject_on_all]
pd.DataFrame(fraction_rejected, columns=['Flag'] + filters)


,Flag,HSC-I,HSC-Y
0,base_SdssCentroid_flag_edge,3%,3%
1,base_SdssCentroid_flag_noSecondDerivative,0%,0%
2,base_SdssCentroid_flag_almostNoSecondDerivative,3%,3%
3,base_SdssCentroid_flag_notAtMaximum,35%,38%
4,base_SdssCentroid_flag_resetToPeak,2%,1%
5,base_CircularApertureFlux_flag_badCentroid,15%,15%
6,base_GaussianFlux_flag_badCentroid,15%,15%
7,base_InputCount_flag_badCentroid,15%,15%
8,base_LocalBackground_flag_badCentroid,15%,15%
9,base_PsfFlux_flag_badCentroid,15%,15%


In [ ]:
#The data products of interest are:
#    good_objects_by_filter = all objects that pass flags for all filters.
#    good_stars_by_filter = all objects that pass flags for all filters and are deemed to be stars. 
#    good_galaxies_by_filter = all objects that pass flags for all filters and are deemed to be galaxies.
#    All data products are lists of astropy tables.  
#    The length and sequences of the lists corresponds to the specified filters.  